In [4]:
!python --version

Python 3.11.13


In [5]:
!mkdir MOAT3
%cd MOAT3

/kaggle/working/MOAT3


In [6]:
!git clone https://github.com/google-research/deeplab2

Cloning into 'deeplab2'...
remote: Enumerating objects: 2252, done.
remote: Counting objects: 100% (1008/1008), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 2252 (delta 847), reused 788 (delta 788), pack-reused 1244 (from 1)
Receiving objects: 100% (2252/2252), 8.42 MiB | 30.68 MiB/s, done.
Resolving deltas: 100% (1598/1598), done.


### Installation

In [95]:
!pip install tensorflow==2.15.0 keras==2.15.0 --force-reinstall

  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.14.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.7 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cache

In [89]:
!pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras


In [9]:
!sudo apt-get install protobuf-compiler -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
!pip install protobuf==3.20.3

In [11]:
!pip install pillow
!pip install matplotlib
!pip install cython

In [ ]:
%cd /kaggle/working/MOAT3
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/cocodataset/cocoapi.git
%cd /kaggle/working/MOAT3/cocoapi/PythonAPI
!make

In [13]:
!mv /kaggle/working/MOAT3/cocoapi /kaggle/working/MOAT3/cocoapi_1

In [14]:
%cd /kaggle/working/MOAT3
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models:`pwd`/cocoapi/PythonAPI

/kaggle/working/MOAT3


In [15]:
%cd /kaggle/working/MOAT3
!protoc deeplab2/*.proto --python_out=.

/kaggle/working/MOAT3


In [ ]:
!pip install numpy==1.26.4 --force-reinstall

In [17]:
import os
os.environ['PYTHONPATH'] = (
    os.environ.get('PYTHONPATH', '') +
    f":{os.getcwd()}:{os.getcwd()}/models:{os.getcwd()}/cocoapi/PythonAPI"
)

In [90]:
!rm -rf /kaggle/working/MOAT3/deeplab2/tensorflow_ops/kernels/merge_semantic_and_instance_maps_op_kernel.cu.o
!rm -rf /kaggle/working/MOAT3/deeplab2/tensorflow_ops/kernels/merge_semantic_and_instance_maps_op.so

In [93]:
!pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras


In [94]:
!ls /kaggle/working/MOAT3/deeplab2/tensorflow_ops/kernels

merge_semantic_and_instance_maps_op.cc
merge_semantic_and_instance_maps_op_kernel.cc
merge_semantic_and_instance_maps_op_kernel.cu.cc
merge_semantic_and_instance_maps_op_kernel.h


In [96]:
%%bash
set -x  # bật chế độ debug để in toàn bộ lệnh

cd /kaggle/working/MOAT3

# Lấy cờ compile/link của TensorFlow
TF_CFLAGS=($(python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_compile_flags()))'))
TF_LFLAGS=($(python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_link_flags()))'))

OP_NAME="deeplab2/tensorflow_ops/kernels/merge_semantic_and_instance_maps_op"

# Bước 1: Compile CUDA kernel
nvcc -v -std=c++14 -c -o ${OP_NAME}_kernel.cu.o ${OP_NAME}_kernel.cu.cc \
${TF_CFLAGS[@]} -D GOOGLE_CUDA=1 -x cu -Xcompiler -fPIC --expt-relaxed-constexpr


# Bước 2: Link thành file .so
g++ -std=c++14 -shared -o ${OP_NAME}.so ${OP_NAME}.cc ${OP_NAME}_kernel.cc \
${OP_NAME}_kernel.cu.o ${TF_CFLAGS[@]} -fPIC -L/usr/local/cuda/lib64 -lcudart ${TF_LFLAGS[@]}

+ cd /kaggle/working/MOAT3
+ TF_CFLAGS=($(python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_compile_flags()))'))
++ python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_compile_flags()))'
2025-08-16 11:33:22.689232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-16 11:33:22.689298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-16 11:33:22.691358: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
+ TF_LFLAGS=($(python3 -c 'import tensorflow as tf; print(" ".join(tf.sysconfig.get_link_flags()))'))
++ python3 -c 'import tensorflow as tf; 

In [97]:
!ls /kaggle/working/MOAT3/deeplab2/tensorflow_ops/kernels

merge_semantic_and_instance_maps_op.cc
merge_semantic_and_instance_maps_op_kernel.cc
merge_semantic_and_instance_maps_op_kernel.cu.cc
merge_semantic_and_instance_maps_op_kernel.cu.o
merge_semantic_and_instance_maps_op_kernel.h
merge_semantic_and_instance_maps_op.so


In [ ]:
%cd /kaggle/working/MOAT3
!python deeplab2/tensorflow_ops/python/kernel_tests/merge_semantic_and_instance_maps_op_test.py

/kaggle/working/MOAT3
2025-08-16 11:36:07.217159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-16 11:36:07.217215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-16 11:36:07.218989: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Running tests under Python 3.11.13: /usr/bin/python3
[ RUN      ] MergeSemanticAndInstanceMapsOpGpuTest.testMergeSemanticAndInstanceMaps
I0816 11:36:10.052149 133760262169728 test_util.py:2574] time(__main__.MergeSemanticAndInstanceMapsOpGpuTest.testMergeSemanticAndInstanceMaps): 0.02s
[       OK ] MergeSemanticAndInstanceMapsOpGpuTest.testM

#Test setup

### Prepare dataset

In [24]:
%cd /kaggle/working/MOAT3/deeplab2/data
!mkdir ade
%cd ade
!curl -L -o ADEChallengeData2016.zip\
  https://www.kaggle.com/api/v1/datasets/download/awsaf49/ade20k-dataset

/kaggle/working/MOAT3/deeplab2/data
/kaggle/working/MOAT3/deeplab2/data/ade
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1124M  100 1124M    0     0   128M      0  0:00:08  0:00:08 --:--:--  128M


In [25]:
%cd /kaggle/working/MOAT3/deeplab2/data/ade
!unzip -q ADEChallengeData2016.zip

/kaggle/working/MOAT3/deeplab2/data/ade


In [27]:
%cd /kaggle/working/MOAT3/deeplab2/data/ade/ADEChallengeData2016/images
!mv training training_sequence 
!mv validation validation_sequence
!mkdir train
!mkdir val
!mv training_sequence train
!mv validation_sequence val

/kaggle/working/MOAT3/deeplab2/data/ade/ADEChallengeData2016/images


In [28]:
%cd /kaggle/working/MOAT3/deeplab2/data/ade/ADEChallengeData2016/annotations
!mv training training_sequence 
!mv validation validation_sequence
!mkdir train
!mkdir val
!mv training_sequence train
!mv validation_sequence val

/kaggle/working/MOAT3/deeplab2/data/ade/ADEChallengeData2016/annotations


In [29]:
%%writefile /kaggle/working/MOAT3/deeplab2/data/dataset.py
# Copyright 2023 The Deeplab2 Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Provides data from segmentation datasets.

Currently, we support the following datasets:

1. Cityscapes dataset (https://www.cityscapes-dataset.com).

The Cityscapes dataset contains 19 semantic labels (such as road, person, car,
and so on) for urban street scenes.


2. KITTI-STEP (http://www.cvlibs.net/datasets/kitti/).

The KITTI-STEP enriches the KITTI-MOTS data with additional `stuff'
anntotations.

3. MOTChallenge-STEP (https://motchallenge.net/).

The MOTChallenge-STEP enriches the MOTSChallenge data with additional `stuff'
annotations.

4. MSCOCO panoptic segmentation (http://cocodataset.org/#panoptic-2018).

Panoptic segmentation annotations for MSCOCO dataset. Note that we convert the
provided MSCOCO panoptic segmentation format to the following one:
panoptic label = semantic label * 256 + instance id.

5. Cityscapes-DVPS (https://github.com/joe-siyuan-qiao/ViP-DeepLab)

The Cityscapes-DVPS dataset augments Cityscapes-VPS
(https://github.com/mcahny/vps) with depth annotations.

6. SemKITTI-DVPS (https://github.com/joe-siyuan-qiao/ViP-DeepLab)

The SemKITTI-DVPS dataset converts 3D point annotations of SemanticKITTI
(http://www.semantic-kitti.org) into 2D image labels.

7. WOD-PVPS
(https://waymo.com/open/data/perception/#2d-video-panoptic-segmentation)

The Waymo Open Dataset: Panoramic Video Panoptic Segmentation contains high
quality panoramic video annotations with time and cross-camera consistency.
The Waymo Open Dataset (WOD): Panoramic Video Panoptic Segmentation (PVPS)
contains high quality panoramic video annotations with time and cross-camera
consistency.

8. ADE20K panoptic segmentation
(https://groups.csail.mit.edu/vision/datasets/ADE20K/)

Panoptic segmentation annotations for ADE20K dataset. Note that we convert the
provided ADE20K panoptic segmentation format to the following one:
panoptic label = semantic label * 1000 + instance id.

We can use the dataset in the following settings:
- In the multicamera setting, an example contains all camera data within a
frame (i.e., the instance correspondence between cameras are guaranteed).
- In the non-multicamera setting, we could still use all the camera frames, but
treat them as individual frames. Datasets variants with a subset of cameras
could be done by discarding the rest of the cameras data when creating the
dataset.

The following variants are provided:
- WOD_PVPS_IMAGE_PANOPTIC_SEG: WOD-PVPS dataset as a single frame panoptic
segmentation dataset.
- WOD_PVPS_IMAGE_PANOPTIC_SEG_MULTICAM: WOD-PVPS dataset as a single frame
panoptic segmentation dataset in the multicamera setting.
- WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG: WOD-PVPS dataset as a depth-aware video
panoptic segmentation dataset. The users could ignore the provided depth
groundtruth and use the dataset for the task of video panoptic segmentation.
- WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_MULTICAM: WOD-PVPS dataset as as a
depth-aware video panoptic segmentation dataset in the multicamera setting. The
users could ignore the provided depth groundtruth and use the dataset for the
task of panoramic video panoptic segmentation.


References:

- Marius Cordts, Mohamed Omran, Sebastian Ramos, Timo Rehfeld, Markus
  Enzweiler, Rodrigo Benenson, Uwe Franke, Stefan Roth, and Bernt Schiele, "The
  Cityscapes Dataset for Semantic Urban Scene Understanding." In CVPR, 2016.

- Andreas Geiger and Philip Lenz and Raquel Urtasun, "Are we ready for
  Autonomous Driving? The KITTI Vision Benchmark Suite." In CVPR, 2012.

- Alexander Kirillov, Kaiming He, Ross Girshick, Carsten Rother, and Piotr
  Dollar, "Panoptic Segmentation." In CVPR, 2019.

- Tsung-Yi Lin, Michael Maire, Serge J. Belongie, Lubomir D. Bourdev, Ross B.
  Girshick, James Hays, Pietro Perona, Deva Ramanan, Piotr Dollar, and C.
  Lawrence Zitnick, "Microsoft COCO: common objects in context." In ECCV, 2014.

- Anton Milan, Laura Leal-Taixe, Ian Reid, Stefan Roth, and Konrad Schindler,
  "Mot16: A benchmark for multi-object tracking." arXiv:1603.00831, 2016.

- Paul Voigtlaender, Michael Krause, Aljosa Osep, Jonathon Luiten, Berin
  Balachandar Gnana Sekar, Andreas Geiger, and Bastian Leibe. "MOTS:
  Multi-object tracking and segmentation." In CVPR, 2019

- Mark Weber, Jun Xie, Maxwell Collins, Yukun Zhu, Paul Voigtlaender, Hartwig
  Adam, Bradley Green, Andreas Geiger, Bastian Leibe, Daniel Cremers, Aljosa
  Osep, Laura Leal-Taixe, and Liang-Chieh Chen, "STEP: Segmenting and Tracking
  Every Pixel." arXiv: 2102.11859, 2021.

- Dahun Kim, Sanghyun Woo, Joon-Young Lee, and In So Kweon. "Video panoptic
  segmentation." In CVPR, 2020.

- Jens Behley, Martin Garbade, Andres Milioto, Jan Quenzel, Sven Behnke, Cyrill
  Stachniss, and Jurgen Gall. "Semantickitti: A dataset for semantic scene
  understanding of lidar sequences." In ICCV, 2019.

- Siyuan Qiao, Yukun Zhu, Hartwig Adam, Alan Yuille, and Liang-Chieh Chen.
  "ViP-DeepLab: Learning Visual Perception with Depth-aware Video Panoptic
  Segmentation." In CVPR, 2021.

- Jieru Mei, Alex Zihao Zhu, Xinchen Yan, Hang Yan, Siyuan Qiao, Yukun Zhu,
  Liang-Chieh Chen, Henrik Kretzschmar, and Dragomir Anguelov. "Waymo Open
  Dataset: Panoramic Video Panoptic Segmentation." arXiv: 2206.07704, 2022.

- Bolei Zhou, Hang Zhao, Xavier Puig, Sanja Fidler, Adela Barriuso,
  and Antonio Torralba. "Scene Parsing Through ADE20K Dataset." In CVPR, 2017.

"""

import collections

from deeplab2.data import ade20k_constants
from deeplab2.data import waymo_constants

# Dataset names.
_CITYSCAPES_PANOPTIC = 'cityscapes_panoptic'
_KITTI_STEP = 'kitti_step'
_MOTCHALLENGE_STEP = 'motchallenge_step'
_CITYSCAPES_DVPS = 'cityscapes_dvps'
_SEMKITTI_DVPS = 'semkitti_dvps'
_COCO_PANOPTIC = 'coco_panoptic'
_ADE20K_PANOPTIC = 'ade20k_panoptic'
_ADE20K_SEMANTIC = 'ade20k_semantic'

# WOD: PVPS dataset names.
_WOD_PVPS_IMAGE_PANOPTIC_SEG = 'wod_pvps_image_panoptic_seg'
_WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG = 'wod_pvps_depth_video_panoptic_seg'
_WOD_PVPS_IMAGE_PANOPTIC_SEG_MULTICAM = 'wod_pvps_image_panoptic_seg_multicam'
_WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_MULTICAM = (
    'wod_pvps_depth_video_panoptic_seg_multicam')


# Colormap names.
CITYSCAPES_COLORMAP = 'cityscapes'
MOTCHALLENGE_COLORMAP = 'motchallenge'
COCO_COLORMAP = 'coco'
_ADE20K_COLORMAP = 'ade20k'
WOD_PVPS_COLORMAP = waymo_constants.COLORMAP

# Camera Names for WOD: PVPS.
WOD_PVPS_CAMERA_NAMES = [
    'side_left', 'front_left', 'front',
    'front_right', 'side_right'
]


# Named tuple to describe dataset properties.
DatasetDescriptor = collections.namedtuple(
    'DatasetDescriptor', [
        'dataset_name',  # Dataset name.
        'splits_to_sizes',  # Splits of the dataset into training, val and test.
        'num_classes',   # Number of semantic classes.
        'ignore_label',  # Ignore label value used for semantic segmentation.

        # Fields below are used for panoptic segmentation and will be None for
        # Semantic segmentation datasets.
        # Label divisor only used in panoptic segmentation annotation to infer
        # semantic label and instance id.
        'panoptic_label_divisor',
        # A tuple of classes that contains instance annotations. For example,
        # 'person' class has instance annotations while 'sky' does not.
        'class_has_instances_list',
        # A flag indicating whether the dataset is a video dataset that contains
        # sequence IDs and frame IDs.
        'is_video_dataset',
        # A string specifying the colormap that should be used for
        # visualization. E.g. 'cityscapes'.
        'colormap',
        # A flag indicating whether the dataset contains depth annotation.
        'is_depth_dataset',
        # The ignore label for depth annotations.
        'ignore_depth',
        # A list of camera names, only for multicamera setup.
        'camera_names',
    ]
)


def _build_dataset_info(**kwargs):
  """Builds dataset information with default values."""
  default = {
      'camera_names': None,
  }
  default.update(kwargs)
  return DatasetDescriptor(**default)


CITYSCAPES_PANOPTIC_INFORMATION = _build_dataset_info(
    dataset_name=_CITYSCAPES_PANOPTIC,
    splits_to_sizes={'train_fine': 2975,
                     'val_fine': 500,
                     'trainval_fine': 3475,
                     'test_fine': 1525},
    num_classes=19,
    ignore_label=255,
    panoptic_label_divisor=1000,
    class_has_instances_list=tuple(range(11, 19)),
    is_video_dataset=False,
    colormap=CITYSCAPES_COLORMAP,
    is_depth_dataset=False,
    ignore_depth=None,
)

KITTI_STEP_INFORMATION = _build_dataset_info(
    dataset_name=_KITTI_STEP,
    splits_to_sizes={'train': 5027,
                     'val': 2981,
                     'test': 11095},
    num_classes=19,
    ignore_label=255,
    panoptic_label_divisor=1000,
    class_has_instances_list=(11, 13),
    is_video_dataset=True,
    colormap=CITYSCAPES_COLORMAP,
    is_depth_dataset=False,
    ignore_depth=None,
)

MOTCHALLENGE_STEP_INFORMATION = _build_dataset_info(
    dataset_name=_MOTCHALLENGE_STEP,
    splits_to_sizes={'train': 525,  # Sequence 9.
                     'val': 600,  # Sequence 2.
                     'test': 0},
    num_classes=7,
    ignore_label=255,
    panoptic_label_divisor=1000,
    class_has_instances_list=(4,),
    is_video_dataset=True,
    colormap=MOTCHALLENGE_COLORMAP,
    is_depth_dataset=False,
    ignore_depth=None,
)

CITYSCAPES_DVPS_INFORMATION = _build_dataset_info(
    dataset_name=_CITYSCAPES_DVPS,
    # The numbers of images are 2400/300/300 for train/val/test. Here, the
    # sizes are the number of consecutive frame pairs. As each sequence has 6
    # frames, the number of pairs for the train split is 2400 / 6 * 5 = 2000.
    # Similarly, we get 250 pairs for the val split and the test split.
    splits_to_sizes={'train': 2000,
                     'val': 250,
                     'test': 250},
    num_classes=19,
    ignore_label=32,
    panoptic_label_divisor=1000,
    class_has_instances_list=tuple(range(11, 19)),
    is_video_dataset=True,
    colormap=CITYSCAPES_COLORMAP,
    is_depth_dataset=True,
    ignore_depth=0,
)

SEMKITTI_DVPS_INFORMATION = _build_dataset_info(
    dataset_name=_SEMKITTI_DVPS,
    splits_to_sizes={'train': 19120,
                     'val': 4070,
                     'test': 4340},
    num_classes=19,
    ignore_label=255,
    panoptic_label_divisor=65536,
    class_has_instances_list=tuple(range(8)),
    is_video_dataset=True,
    # Reuses Cityscapes colormap.
    colormap=CITYSCAPES_COLORMAP,
    is_depth_dataset=True,
    ignore_depth=0,
)

COCO_PANOPTIC_INFORMATION = _build_dataset_info(
    dataset_name=_COCO_PANOPTIC,
    splits_to_sizes={'train': 118287,
                     'val': 5000,
                     'test': 40670},
    num_classes=134,
    ignore_label=0,
    panoptic_label_divisor=256,
    class_has_instances_list=tuple(range(1, 81)),
    is_video_dataset=False,
    colormap=COCO_COLORMAP,
    is_depth_dataset=False,
    ignore_depth=None,
)

ADE20K_PANOPTIC_INFORMATION = _build_dataset_info(
    dataset_name=_ADE20K_PANOPTIC,
    splits_to_sizes={
        'train': 20210,
        'val': 2000,
    },
    num_classes=151,
    ignore_label=0,
    panoptic_label_divisor=1000,
    class_has_instances_list=(
        ade20k_constants.get_ade20k_class_has_instances_list()),
    is_video_dataset=False,
    colormap=_ADE20K_COLORMAP,
    is_depth_dataset=False,
    ignore_depth=None,
)

ADE20K_SEMANTIC_INFORMATION = _build_dataset_info(
    dataset_name=_ADE20K_SEMANTIC,
    splits_to_sizes={
        'train': 20210,
        'val': 2000,
    },
    num_classes=150,  # ADE20K semantic segmentation có 150 class
    ignore_label=0,
    panoptic_label_divisor=1000,
    class_has_instances_list = [],
    is_video_dataset=False,
    colormap=_ADE20K_COLORMAP,
    is_depth_dataset=False,
    ignore_depth=None,
)


MAP_NAME_TO_DATASET_INFO = {
    _CITYSCAPES_PANOPTIC: CITYSCAPES_PANOPTIC_INFORMATION,
    _KITTI_STEP: KITTI_STEP_INFORMATION,
    _MOTCHALLENGE_STEP: MOTCHALLENGE_STEP_INFORMATION,
    _CITYSCAPES_DVPS: CITYSCAPES_DVPS_INFORMATION,
    _COCO_PANOPTIC: COCO_PANOPTIC_INFORMATION,
    _ADE20K_PANOPTIC: ADE20K_PANOPTIC_INFORMATION,
    _SEMKITTI_DVPS: SEMKITTI_DVPS_INFORMATION,
    _ADE20K_SEMANTIC: ADE20K_SEMANTIC_INFORMATION,
}

MAP_NAMES = list(MAP_NAME_TO_DATASET_INFO.keys())


def _build_waymo_image_panoptic_seg_dataset(**kwargs):
  """Builds Waymo dataset with default values."""
  waymo_meta = waymo_constants.get_waymo_meta()
  default = dict(
      dataset_name=None,
      # TODO(jierumei): Provide exact size.
      splits_to_sizes={
          'train': 70000,
          'val': 10000,
          'test': 20000,
      },
      ignore_label=waymo_constants.IGNORE_LABEL,
      panoptic_label_divisor=waymo_constants.PANOPTIC_LABEL_DIVISOR,
      num_classes=len(waymo_meta),
      class_has_instances_list=list(
          map(lambda val: val['id'],
              filter(lambda val: val['isthing'], waymo_meta))),
      is_video_dataset=True,
      colormap=waymo_constants.COLORMAP,
      is_depth_dataset=False,
      ignore_depth=None,
      camera_names=None,
  )
  for key in kwargs:
    if key not in default:
      raise ValueError(f'Unknown dataset option: {key}')
  default.update(kwargs)
  return DatasetDescriptor(**default)


WOD_PVPS_IMAGE_PANOPTIC_SEG_DATASET = _build_waymo_image_panoptic_seg_dataset()
WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_DATASET = (
    _build_waymo_image_panoptic_seg_dataset(
        dataset_name=_WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG,
        is_depth_dataset=True,
        ignore_depth=0))
WOD_PVPS_IMAGE_PANOPTIC_SEG_MULTICAM_DATASET = (
    _build_waymo_image_panoptic_seg_dataset(
        dataset_name=_WOD_PVPS_IMAGE_PANOPTIC_SEG_MULTICAM,
        camera_names=WOD_PVPS_CAMERA_NAMES + ['panorama']))
WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_MULTICAM_DATASET = (
    _build_waymo_image_panoptic_seg_dataset(
        dataset_name=_WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_MULTICAM,
        is_depth_dataset=True,
        ignore_depth=0,
        camera_names=WOD_PVPS_CAMERA_NAMES + ['panorama']))

waymo_dataset_all = {
    _WOD_PVPS_IMAGE_PANOPTIC_SEG:
        WOD_PVPS_IMAGE_PANOPTIC_SEG_DATASET,
    _WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG:
        WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_DATASET,
    _WOD_PVPS_IMAGE_PANOPTIC_SEG_MULTICAM:
        WOD_PVPS_IMAGE_PANOPTIC_SEG_MULTICAM_DATASET,
    _WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_MULTICAM:
        WOD_PVPS_DEPTH_VIDEO_PANOPTIC_SEG_MULTICAM_DATASET,
}
MAP_NAME_TO_DATASET_INFO.update(waymo_dataset_all)
MAP_NAMES.extend(list(waymo_dataset_all.keys()))


Overwriting /kaggle/working/MOAT3/deeplab2/data/dataset.py


In [30]:
%%writefile /kaggle/working/MOAT3/deeplab2/data/build_step_data.py
# Copyright 2023 The Deeplab2 Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

r"""Converts STEP (KITTI-STEP or MOTChallenge-STEP) data to sharded TFRecord file format with tf.train.Example protos.

The expected directory structure of the STEP dataset should be as follows:

  + {KITTI | MOTChallenge}-STEP
    + images
       + train
         + sequence_id
           - *.{png|jpg}
           ...
       + val
       + test
    + panoptic_maps
       + train
         + sequence_id
           - *.png
           ...
       + val

The ground-truth panoptic map is encoded as the following in PNG format:

  R: semantic_id
  G: instance_id // 256
  B: instance % 256

See ./utils/create_step_panoptic_maps.py for more details of how we create the
panoptic map by merging semantic and instance maps.

The output Example proto contains the following fields:

  image/encoded: encoded image content.
  image/filename: image filename.
  image/format: image file format.
  image/height: image height.
  image/width: image width.
  image/channels: image channels.
  image/segmentation/class/encoded: encoded panoptic segmentation content.
  image/segmentation/class/format: segmentation encoding format.
  video/sequence_id: sequence ID of the frame.
  video/frame_id: ID of the frame of the video sequence.

The output panoptic segmentation map stored in the Example will be the raw bytes
of an int32 panoptic map, where each pixel is assigned to a panoptic ID:

  panoptic ID = semantic ID * label divisor (1000) + instance ID

where semantic ID will be the same with `category_id` (use TrainId) for
each segment, and ignore label for pixels not belong to any segment.

The instance ID will be 0 for pixels belonging to
  1) `stuff` class
  2) `thing` class with `iscrowd` label
  3) pixels with ignore label
and [1, label divisor) otherwise.

Example to run the scipt:

   python deeplab2/data/build_step_data.py \
     --step_root=${STEP_ROOT} \
     --output_dir=${OUTPUT_DIR}
"""

import math
import os

from typing import Iterator, Sequence, Tuple, Optional

from absl import app
from absl import flags
from absl import logging
import numpy as np

from PIL import Image

import tensorflow as tf

from deeplab2.data import data_utils

FLAGS = flags.FLAGS

flags.DEFINE_string('step_root', None, 'STEP dataset root folder.')

flags.DEFINE_string('output_dir', None,
                    'Path to save converted TFRecord of TensorFlow examples.')
flags.DEFINE_bool(
    'use_two_frames', False, 'Flag to separate between 1 frame '
    'per TFExample or 2 consecutive frames per TFExample.')

_PANOPTIC_LABEL_FORMAT = 'raw'
_NUM_SHARDS = 100
_IMAGE_FOLDER_NAME = 'images'
_PANOPTIC_MAP_FOLDER_NAME = 'annotations'
_LABEL_MAP_FORMAT = 'png'
_INSTANCE_LABEL_DIVISOR = 1000
_ENCODED_INSTANCE_LABEL_DIVISOR = 256
_TF_RECORD_PATTERN = '%s-%05d-of-%05d.tfrecord'
_FRAME_ID_PATTERN = '%06d'


def _get_image_info_from_path(image_path: str) -> Tuple[str, str]:
  """Gets image info including sequence id and image id.

  Image path is in the format of '.../split/sequence_id/image_id.png',
  where `sequence_id` refers to the id of the video sequence, and `image_id` is
  the id of the image in the video sequence.

  Args:
    image_path: Absolute path of the image.

  Returns:
    sequence_id, and image_id as strings.
  """
  sequence_id = image_path.split('/')[-2]
  image_id = os.path.splitext(os.path.basename(image_path))[0]
  return sequence_id, image_id


def _get_images_per_shard(step_root: str, dataset_split: str,
                          sharded_by_sequence: bool) -> Iterator[Sequence[str]]:
  """Gets files for the specified data type and dataset split.

  Args:
    step_root: String, Path to STEP dataset root folder.
    dataset_split: String, dataset split ('train', 'val', 'test')
    sharded_by_sequence: Whether the images should be sharded by sequence or
      even split.

  Yields:
    A list of sorted file lists. Each inner list corresponds to one shard and is
    a list of files for this shard.
  """
  search_files = os.path.join(step_root, _IMAGE_FOLDER_NAME, dataset_split, '*',
                              '*')
  filenames = sorted(tf.io.gfile.glob(search_files))
  num_per_even_shard = int(math.ceil(len(filenames) / _NUM_SHARDS))

  sequence_ids = [os.path.basename(os.path.dirname(name)) for name in filenames]
  images_per_shard = []
  for i, name in enumerate(filenames):
    images_per_shard.append(name)
    shard_data = (i == len(filenames) - 1)
    # Sharded by sequence id.
    shard_data = shard_data or (sharded_by_sequence and
                                sequence_ids[i + 1] != sequence_ids[i])
    # Sharded evenly.
    shard_data = shard_data or (not sharded_by_sequence and
                                len(images_per_shard) == num_per_even_shard)
    if shard_data:
      yield images_per_shard
      images_per_shard = []


def _decode_panoptic_map(panoptic_map_path: str) -> Optional[str]:
    """Decodes grayscale semantic map into panoptic map format."""
    if not tf.io.gfile.exists(panoptic_map_path):
        return None
    with tf.io.gfile.GFile(panoptic_map_path, 'rb') as f:
        panoptic_map = np.array(Image.open(f)).astype(np.int32)

    if panoptic_map.ndim == 2:
        # Ảnh grayscale: mỗi pixel là semantic_id
        semantic_map = panoptic_map
        instance_map = np.zeros_like(semantic_map, dtype=np.int32)
    else:
        # Ảnh panoptic 3 kênh
        semantic_map = panoptic_map[:, :, 0]
        instance_map = (
            panoptic_map[:, :, 1] * _ENCODED_INSTANCE_LABEL_DIVISOR +
            panoptic_map[:, :, 2]
        )

    # Tạo panoptic_id từ semantic_id và instance_id
    panoptic_map = semantic_map * _INSTANCE_LABEL_DIVISOR + instance_map
    return panoptic_map.tobytes()



def _get_previous_frame_path(image_path: str) -> str:
  """Gets previous frame path. If not exists, duplicate it with image_path."""
  frame_id, frame_ext = os.path.splitext(os.path.basename(image_path))
  folder_dir = os.path.dirname(image_path)
  prev_frame_id = _FRAME_ID_PATTERN % (int(frame_id) - 1)
  prev_image_path = os.path.join(folder_dir, prev_frame_id + frame_ext)
  # If first frame, duplicates it.
  if not tf.io.gfile.exists(prev_image_path):
    tf.compat.v1.logging.warn(
        'Could not find previous frame %s of frame %d, duplicate the previous '
        'frame with the current frame.', prev_image_path, int(frame_id))
    prev_image_path = image_path
  return prev_image_path


def _create_panoptic_tfexample(image_path: str,
                               panoptic_map_path: str,
                               use_two_frames: bool,
                               is_testing: bool = False) -> tf.train.Example:
    """Creates a TF example for each image."""
    with tf.io.gfile.GFile(image_path, 'rb') as f:
        image_data = f.read()
    label_data = None
    if not is_testing:
        label_data = _decode_panoptic_map(panoptic_map_path)
    image_name = os.path.basename(image_path)

    # Fix lỗi JPG -> JPEG
    image_format = image_name.split('.')[-1].lower()
    if image_format == 'jpg':
        image_format = 'jpeg'

    sequence_id, frame_id = _get_image_info_from_path(image_path)
    prev_image_data = None
    prev_label_data = None
    if use_two_frames:
        # Previous image
        prev_image_path = _get_previous_frame_path(image_path)
        with tf.io.gfile.GFile(prev_image_path, 'rb') as f:
            prev_image_data = f.read()
        # Previous panoptic map
        if not is_testing:
            prev_panoptic_map_path = _get_previous_frame_path(panoptic_map_path)
            prev_label_data = _decode_panoptic_map(prev_panoptic_map_path)

    return data_utils.create_video_tfexample(
        image_data,
        image_format,
        image_name,
        label_format=_PANOPTIC_LABEL_FORMAT,
        sequence_id=sequence_id,
        image_id=frame_id,
        label_data=label_data,
        prev_image_data=prev_image_data,
        prev_label_data=prev_label_data
    )

def _convert_dataset(step_root: str,
                     dataset_split: str,
                     output_dir: str,
                     use_two_frames: bool = False):
  """Converts the specified dataset split to TFRecord format.

  Args:
    step_root: String, Path to STEP dataset root folder.
    dataset_split: String, the dataset split (e.g., train, val).
    output_dir: String, directory to write output TFRecords to.
    use_two_frames: Whether to encode consecutive two frames in the Example.
  """
  # For val and test set, if we run with use_two_frames, we should create a
  # sorted tfrecord per sequence.
  create_tfrecord_per_sequence = ('train'
                                  not in dataset_split) and use_two_frames
  is_testing = 'test' in dataset_split

  image_files_per_shard = list(
      _get_images_per_shard(step_root, dataset_split,
                            sharded_by_sequence=create_tfrecord_per_sequence))
  num_shards = len(image_files_per_shard)

  for shard_id, image_list in enumerate(image_files_per_shard):
    shard_filename = _TF_RECORD_PATTERN % (dataset_split, shard_id, num_shards)
    output_filename = os.path.join(output_dir, shard_filename)
    with tf.io.TFRecordWriter(output_filename) as tfrecord_writer:
      for image_path in image_list:
        sequence_id, image_id = _get_image_info_from_path(image_path)
        panoptic_map_path = os.path.join(
            step_root, _PANOPTIC_MAP_FOLDER_NAME, dataset_split, sequence_id,
            '%s.%s' % (image_id, _LABEL_MAP_FORMAT))
        example = _create_panoptic_tfexample(image_path, panoptic_map_path,
                                             use_two_frames, is_testing)
        tfrecord_writer.write(example.SerializeToString())


def main(argv: Sequence[str]) -> None:
  if len(argv) > 1:
    raise app.UsageError('Too many command-line arguments.')
  tf.io.gfile.makedirs(FLAGS.output_dir)
  for dataset_split in ('train', 'val'):
    logging.info('Starts to processing STEP dataset split %s.', dataset_split)
    _convert_dataset(FLAGS.step_root, dataset_split, FLAGS.output_dir,
                     FLAGS.use_two_frames)


if __name__ == '__main__':
  app.run(main)


Overwriting /kaggle/working/MOAT3/deeplab2/data/build_step_data.py


In [31]:
!mkdir /kaggle/tmp

In [32]:
%cd /kaggle/working/MOAT3/deeplab2
!python data/build_step_data.py \
--step_root   /kaggle/working/MOAT3/deeplab2/data/ade/ADEChallengeData2016 \
--output_dir  /kaggle/tmp/ADEChallengeData2016_tfrecord

/kaggle/working/MOAT3/deeplab2
2025-08-16 10:15:34.695184: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-16 10:15:34.695240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-16 10:15:34.696880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0816 10:15:37.834607 139205893121152 build_step_data.py:291] Starts to processing STEP dataset split train.
I0816 10:17:46.886661 139205893121152 build_step_data.py:291] Starts to processing STEP dataset split val.


In [ ]:
!ls /kaggle/tmp/ADEChallengeData2016_tfrecord

### Download pretrained weights


In [34]:
%cd /kaggle/working/MOAT3/deeplab2
!mkdir pretrained
%cd pretrained
!wget https://storage.googleapis.com/gresearch/tf-deeplab/checkpoint/moat3_imagenet_22k_and_1k_512_wo_position_embedding.tar.gz

/kaggle/working/MOAT3/deeplab2
/kaggle/working/MOAT3/deeplab2/pretrained
--2025-08-16 10:18:16--  https://storage.googleapis.com/gresearch/tf-deeplab/checkpoint/moat3_imagenet_22k_and_1k_512_wo_position_embedding.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.207, 142.251.188.207, 142.250.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 704705442 (672M) [application/x-tar]
Saving to: ‘moat3_imagenet_22k_and_1k_512_wo_position_embedding.tar.gz’

moat3_imagenet_22k_ 100%[===================>] 672.06M   229MB/s    in 2.9s    

2025-08-16 10:18:20 (229 MB/s) - ‘moat3_imagenet_22k_and_1k_512_wo_position_embedding.tar.gz’ saved [704705442/704705442]



In [35]:
%cd /kaggle/working/MOAT3/deeplab2/pretrained
!tar -xzvf moat3_imagenet_22k_and_1k_512_wo_position_embedding.tar.gz

/kaggle/working/MOAT3/deeplab2/pretrained
./moat3_imagenet_22k_and_1k_512_wo_position_embedding/
./moat3_imagenet_22k_and_1k_512_wo_position_embedding/model-ckpt-0.data-00000-of-00001
./moat3_imagenet_22k_and_1k_512_wo_position_embedding/model-ckpt-0.index


In [61]:
!ls /kaggle/working/MOAT3/deeplab2/pretrained/moat3_imagenet_22k_and_1k_512_wo_position_embedding

model-ckpt-0.data-00000-of-00001  model-ckpt-0.index


In [37]:
!mkdir  /kaggle/working/MOAT3/deeplab2/configs/ade20k/moat3

In [65]:
%%writefile /kaggle/working/MOAT3/deeplab2/configs/ade20k/moat3/moat3_large_os32_res321.textproto
# proto-file: deeplab2/config.proto
# proto-message: ExperimentOptions
#
# kMaX-DeepLab with ConvNeXt-large backbone with output stride 32.
#
############### PLEASE READ THIS BEFORE USING THIS CONFIG ###############
# Before using this config, you need to update the following fields:
# - experiment_name: Use a unique experiment name for each experiment.
# - pretrained_weights: Update the path to the pretrained weights.
# - train_dataset_options.file_pattern: Update the path to the
#   training set. e.g., your_dataset/train*.tfrecord
# - eval_dataset_options.file_pattern: Update the path to the
#   validation set, e.g., your_dataset/eval*.tfrecord
#########################################################################
#
# References:
# For ConvNeXt, see
# - Zhuang Liu, et al. "A ConvNet for the 2020s." In CVPR, 2022.
# For kMaX-DeepLab, see
# - Qihang Yu, et al. "k-means Mask Transformer." In ECCV, 2022.

# Use a unique experiment_name for each experiment.
experiment_name: "moat3_finetune_512_22k_ade20k_semantic"
model_options {
  backbone {
    # kMaX meta is used with the prefix "kmax_".
    name: "kmax_moat3_finetune_512_no_pe_22k"
    drop_path_keep_prob: 0.3
    # We use pretrained_weights instead of initial_checkpoint for loading
    # specifical weights for backbone (e.g., converted pytorch weights).
    # pretrained_weights: "pretrained/moat3_imagenet_22k_and_1k_512_wo_position_embedding/model-ckpt-0"
  }
  decoder {
    feature_key: "feature_semantic"
    decoder_channels: 256
    aspp_channels: 256
    atrous_rates: 6
    atrous_rates: 12
    atrous_rates: 18
  }
  max_deeplab {
    pixel_space_head {
      output_channels: 128
      head_channels: 256
    }
    auxiliary_low_level {
      feature_key: "res3"
      channels_project: 64
    }
    auxiliary_low_level {
      feature_key: "res2"
      channels_project: 32
    }
    auxiliary_semantic_head {
      output_channels: 150
      head_channels: 256
    }
  }
}
trainer_options {
  save_checkpoints_steps: 400
  save_summaries_steps: 100
  steps_per_loop: 100
  loss_options {
    semantic_loss {
      name: "softmax_cross_entropy"
      weight: 1.0
    }
  }
  solver_options {
    base_learning_rate: 0.0005
    training_number_of_steps: 100000
    warmup_steps: 5000
    backbone_learning_rate_multiplier: 0.1
    optimizer: "adamw"
    adamw_weight_decay: 0.05
  }
}
train_dataset_options {
  dataset: "ade20k_semantic"
  file_pattern: "/kaggle/tmp/ADEChallengeData2016_tfrecord/train*.tfrecord"
  # Adjust the batch_size accordingly to better fit your GPU/TPU memory.
  # Also see Q1 in g3doc/faq.md.
  batch_size: 2
  crop_size: 321
  crop_size: 321
  min_resize_value: 321
  max_resize_value: 321
  augmentations {
    min_scale_factor: 0.5
    max_scale_factor: 2.0
    scale_factor_step_size: 0.1
    # autoaugment_policy_name: "simple_classification_policy"
  }
}
eval_dataset_options {
  dataset: "ade20k_semantic"
  # Update the path to validation set.
  file_pattern: "/kaggle/tmp/ADEChallengeData2016_tfrecord/val*.tfrecord"
  batch_size: 1
  crop_size: 321
  crop_size: 321
  min_resize_value: 321
  max_resize_value: 321
}
evaluator_options {
  continuous_eval_timeout: -1
  save_predictions: true
  save_raw_predictions: false
  eval_interval: 400
}

Overwriting /kaggle/working/MOAT3/deeplab2/configs/ade20k/moat3/moat3_large_os32_res321.textproto


In [71]:
# %load /kaggle/working/MOAT3/deeplab2/trainer/trainer.py
# Copyright 2023 The Deeplab2 Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""This file contains code to create a Trainer for training and validation."""

from typing import Dict, Any, Text
import orbit
import tensorflow as tf

from deeplab2 import common
from deeplab2 import config_pb2
from deeplab2.model import utils
from deeplab2.trainer import runner_utils
from deeplab2.trainer import trainer_utils


def _create_optimizer(
    solver_config: config_pb2.SolverOptions,
    learning_rate_multiplier: float = 1.0) -> tf.keras.optimizers.Optimizer:
  """Creates an Optimizer based on the configuration.

  Args:
    solver_config: A trainer_pb2.SolverOptions configuration.
    learning_rate_multiplier: A float, the learning rate multiplier applied on
      top of the base learning rate. Default to 1.0.

  Returns:
    A tf.keras.optimizer.Optimizer.

  Raises:
    ValueError: An error occurs when the desired optimizer or learning rate
      scheduler is not supported.
  """
  learning_rate = (solver_config.base_learning_rate * learning_rate_multiplier)
  if solver_config.learning_policy == 'poly':
    lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=learning_rate,
        decay_steps=solver_config.training_number_of_steps,
        end_learning_rate=solver_config.poly_end_learning_rate,
        power=solver_config.poly_learning_power,
        cycle=False)
  elif solver_config.learning_policy == 'cosine':
    lr_scheduler = tf.keras.experimental.CosineDecay(
        initial_learning_rate=learning_rate,
        decay_steps=solver_config.training_number_of_steps,
        alpha=0.0)
  else:
    raise ValueError('Learning rate policy %s is not supported.' %
                     solver_config.learning_policy)

  if solver_config.warmup_steps:
    lr_scheduler = trainer_utils.WarmUp(
        initial_learning_rate=learning_rate,
        decay_schedule_fn=lr_scheduler,
        warmup_steps=solver_config.warmup_steps,
        name='linear_warmup')

  if solver_config.optimizer == 'adam':
    return tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
  elif solver_config.optimizer == 'sgd':
    # We use momentum = 0.9, the most frequently used case.
    return tf.keras.optimizers.SGD(learning_rate=lr_scheduler,
                                   momentum=0.9)
  elif solver_config.optimizer == 'adamw':
    return trainer_utils.AdamWeightDecay(
        learning_rate=lr_scheduler,
        weight_decay_rate=solver_config.adamw_weight_decay,
        # Weight decay is only applied to convolution/linear kernels/weights.
        include_in_weight_decay=[r'.*(kernel|weight):0$'],
        exclude_from_weight_decay=[r'.*$'],
        gradient_clip_norm=0.0)

  raise ValueError('Optimizer %s is not supported.' % solver_config.optimizer)


class Trainer(orbit.StandardTrainer):
  """Implements a Trainer for training DeepLab models."""

  def __init__(self, config: config_pb2.ExperimentOptions,
               model: tf.keras.Model, loss: tf.keras.losses.Loss,
               global_step: tf.Variable):
    """Initializes the trainer.

    Args:
      config: A config_pb2.ExperimentOptions configuration.
      model: A tf.keras.Model.
      loss: A tf.keras.losses.Loss.
      global_step: A tf.Variable that records the global training step.
    """
    self._strategy = tf.distribute.get_strategy()

    support_panoptic = (common.TASK_PANOPTIC_SEGMENTATION in
                        utils.get_supported_tasks(config))
    train_dataset = runner_utils.create_dataset(
        config.train_dataset_options,
        is_training=True,
        only_semantic_annotations=not support_panoptic)
    train_dataset = orbit.utils.make_distributed_dataset(
        self.strategy, train_dataset)
    super(Trainer, self).__init__(train_dataset)

    self._config = config
    self._model = model
    self._loss = loss

    solver_options = config.trainer_options.solver_options
    self._optimizer = _create_optimizer(solver_options)
    self._backbone_optimizer = None
    if solver_options.HasField('backbone_learning_rate_multiplier'):
      self._backbone_optimizer = _create_optimizer(
          solver_options, learning_rate_multiplier=(
              solver_options.backbone_learning_rate_multiplier))

    self._global_step = global_step
    self._use_gradient_clipping = solver_options.use_gradient_clipping
    self._clip_gradient_norm = solver_options.clip_gradient_norm

    self._train_loss_metric_dict = runner_utils.create_loss_metric_dict(
        loss.get_loss_names(), prefix='train_')

  def train_loop_begin(self):
    """Called once at the beginning of the training loop.

    This method is called before dataset iterators creation.
    """
    for metric in self._train_loss_metric_dict.values():
      metric.reset_states()

  def _apply_gradients_to_optimizers(self, gradients_and_variables):
    """Applies gradients to their optimizers.

    This function divides all trainable variables (and their gradients) into
    two groups. One group contains backbone variables that have been pretrained,
    e.g., on ImageNet classification. The other group contains all other
    variables that are added specifically for the dense prediction task, e.g.,
    panoptic segmentation. Then, we apply two optimizers, optionally with two
    learning rates, to the variables and gradients.

    Args:
      gradients_and_variables: A list of tuple of (gradient, variable) tensors.
    """
    if self._backbone_optimizer is None:
      self._optimizer.apply_gradients(gradients_and_variables)
    else:
      optimizer_inputs = []
      backbone_optimizer_inputs = []

      encoder = self._model.checkpoint_items['encoder']
      encoder_variable_names = [x.name for x in encoder.trainable_variables]
      encoder_name = self._config.model_options.backbone.name

      for gradient, variable in gradients_and_variables:
        if runner_utils.check_if_variable_in_backbone(variable, encoder_name,
                                                      encoder_variable_names):
          backbone_optimizer_inputs.append((gradient, variable))
        else:
          optimizer_inputs.append((gradient, variable))
      self._optimizer.apply_gradients(optimizer_inputs)
      self._backbone_optimizer.apply_gradients(backbone_optimizer_inputs)

  def train_step(self, iterator):
    """Implements one step of training.

    Runs one step of evaluation with respect to the chosen strategy. In case of
    a distributed strategy, the replica results are gathered and returned.

    Note that all operations within `_train_step` are tf.function compatible, as
    they will be traced with tf.function. Any other/numpy operations are put in
    `train_loop_begin` or `train_loop_end` functions.

    Args:
      iterator: A tf.nest-compatible structure of tf.data Iterator or
        DistributedIterator.
    """

    def step_fn(inputs):
      self._train_step(inputs)
      self._global_step.assign_add(1)

    self._strategy.run(step_fn, args=(next(iterator),))

  def _train_step(self, inputs: Dict[Text, Any]):
    """Performs a forward and backward pass.

    Args:
      inputs: A dictionary to be consumed by the model.
    """
    with tf.GradientTape() as tape:
      outputs = self._model(inputs[common.IMAGE], training=True)
      # Get the average per-batch loss and scale it down by the number of
      # replicas. This ensures that we don't end up multiplying our loss by the
      # number of workers - gradients are summed, not averaged, across replicas
      # during the apply_gradients call.
      loss_dict = self._loss(inputs, outputs)
      # Average over the batch.
      average_loss_dict = {}
      for name, loss in loss_dict.items():
        averaged_loss = tf.reduce_mean(loss)
        average_loss_dict[name] = tf.where(tf.math.is_nan(averaged_loss),
                                           0.0, averaged_loss)

      total_loss = average_loss_dict[common.TOTAL_LOSS]
      scaled_loss = total_loss / self.strategy.num_replicas_in_sync

    training_vars = self._model.trainable_variables
    gradients = tape.gradient(scaled_loss, training_vars)

    # Apply gradient clipping.
    if self._clip_gradient_norm > 0.0 and self._use_gradient_clipping:
      gradients, _ = tf.clip_by_global_norm(gradients, self._clip_gradient_norm)

    self._apply_gradients_to_optimizers(list(zip(gradients, training_vars)))

    for name, value in average_loss_dict.items():
      self._train_loss_metric_dict[name].update_state(value)

  def train_loop_end(self) -> Dict[Text, tf.Tensor]:
    """Called at the end of the training loop.

    The value returned from this function will be returned as-is from the
    train() method.

    Returns:
      A dictionary of `Tensors`, which will be written to logs and as
      TensorBoard summaries.
    """
    train_logs = {}
    for loss_metric in self._train_loss_metric_dict.values():
      train_logs['losses/' + loss_metric.name] = loss_metric.result()

    if callable(self._optimizer.learning_rate):
      train_logs['learning_rate'] = self._optimizer.learning_rate(
          self._global_step)
    else:
      train_logs['learning_rate'] = self._optimizer.learning_rate
    return train_logs

  @property
  def optimizer(self):
    return self._optimizer

  @property
  def backbone_optimizer(self):
    return self._backbone_optimizer

  @property
  def strategy(self):
    return self._strategy

  @property
  def global_step(self):
    return self._global_step

  @property
  def model(self):
    return self._model


In [76]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%cd /kaggle/working/MOAT3/deeplab2/
!python trainer/train.py \
    --config_file=configs/ade20k/moat3/moat3_large_os32_res321.textproto \
    --mode=train_and_eval \
    --model_dir=moat3_large_os32_res321_workdirs \
    --num_gpus=1
    # > train_log.txt 2>&1


In [85]:
!nvcc  --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [69]:
# %load train_log.txt
2025-08-16 11:00:33.296683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-16 11:00:33.296754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-16 11:00:33.298549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0816 11:00:37.064345 137353541276800 train.py:65] Reading the config file.
I0816 11:00:37.066242 137353541276800 train.py:69] Starting the experiment.
I0816 11:00:37.068083 137353541276800 mirrored_strategy.py:423] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
I0816 11:00:37.088314 137353541276800 train_lib.py:108] Using strategy <class 'tensorflow.python.distribute.mirrored_strategy.MirroredStrategy'> with 2 replicas
Traceback (most recent call last):
  File "/kaggle/working/MOAT3/deeplab2/trainer/train.py", line 76, in <module>
    app.run(main)
  File "/usr/local/lib/python3.11/dist-packages/absl/app.py", line 316, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.11/dist-packages/absl/app.py", line 261, in _run_main
    sys.exit(main(argv))
             ^^^^^^^^^^
  File "/kaggle/working/MOAT3/deeplab2/trainer/train.py", line 71, in main
    train_lib.run_experiment(FLAGS.mode, config, combined_model_dir, FLAGS.master,
  File "/kaggle/working/MOAT3/deeplab2/trainer/train_lib.py", line 118, in run_experiment
    raise ValueError(
ValueError: Using more than one GPU for evaluation is not supported.
